# 数值计算基础

### 原则1：矩阵相乘，小维度优先

矩阵 $A_{n\times p}$，向量 $x_{p\times 1}$，计算 $A^{T}Ax$。

In [1]:
import numpy as np
np.set_printoptions(linewidth=100)

np.random.seed(123)
n = 2000
p = 1000
A = np.random.normal(size=(n, p))
x = np.random.normal(size=p)

方法1：先计算 $A^{T}A$，再与 $x$ 相乘：

In [2]:
%timeit A.transpose().dot(A).dot(x)

13.9 ms ± 462 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


方法2：先计算 $Ax$，再左乘 $A^{T}$：

In [3]:
%timeit A.transpose().dot(A.dot(x))
#和向量有关的计算类似于reduce的一个过程，先计算这部分，压缩维度

424 µs ± 1.18 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


经验法则：对于更一般的矩阵乘法 $A_{m\times n}B_{n\times p}C_{p\times r}$，如果 $n\approx p$ 且 $m>r$，则优先计算 $BC$，反之优先计算 $AB$。

In [4]:
np.random.seed(123)
m = 1000
n = 500
p = 200
r = 100
A = np.random.normal(size=(m, n))
B = np.random.normal(size=(n, p))
C = np.random.normal(size=(p, r))

In [5]:
%timeit A.dot(B).dot(C)

2.71 ms ± 108 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
%timeit A.dot(B.dot(C))

1.55 ms ± 27 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### 原则2：尽量避免显式矩阵求逆!!!!!!!!

矩阵 $A_{n\times n}$，向量 $b_{n\times 1}$，计算 $A^{-1}b$。

In [7]:
np.random.seed(123)
n = 1000
A = np.random.normal(size=(n, n))
b = np.random.normal(size=n)

方法1：先计算 $A^{-1}$，再与 $b$ 相乘：

In [8]:
%timeit np.linalg.inv(A).dot(b)

62.7 ms ± 16.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


方法2：解线性方程组 $Ax=b$：

In [9]:
%timeit np.linalg.solve(A, b)

36 ms ± 6.97 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


线性方程组右边也可以是矩阵，即 $A_{n\times n}$，$B_{n\times p}$，计算 $A^{-1}B$。

In [10]:
np.random.seed(123)
n = 1000
p = 100
A = np.random.normal(size=(n, n))
B = np.random.normal(size=(n, p))

In [11]:
%timeit np.linalg.inv(A).dot(B)

49.4 ms ± 858 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%timeit np.linalg.solve(A, B)

48.4 ms ± 6.95 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 原则3：利用矩阵的特殊结构

矩阵 $A_{n\times n}$，对角矩阵 $W_{n\times n}$，计算 $WA$ 和 $AW$。

In [13]:
np.random.seed(123)
n = 1000
A = np.random.normal(size=(n, n))
w = np.random.normal(size=n)
W = np.diag(w)
#W的有效信息只有对角元素 甚至在储存时可以避免存下完整的对角矩阵 其空间为n^2

In [14]:
%timeit W.dot(A)
#n^3

11.7 ms ± 106 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
%timeit A.dot(W)

12 ms ± 576 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


令 $w_{n\times 1}$ 表示 $W$ 的对角元素，$WA$ 相当于将 $A$ 的每一列乘以 $w$，$AW$ 相当于将 $A$ 的每一行乘以 $w^{T}$。此时可利用 Numpy 的广播机制进行运算。

In [16]:
%timeit A * w.reshape(n, 1)
#n^2

2.04 ms ± 19.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
%timeit A * w

2.17 ms ± 72.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
#A B
#A*b n^2 A*B n^3 A^(-1) n^3 A^(-1)*b n^3但较直接求逆较快 d(A) n^3 ||A||_F^2 n^2 A+b*1'(相当于将b广播)
#

### 尽可能将多个向量运算的循环合并为矩阵运算
理论上复杂度相同 但软件层面对于矩阵运算优化更好       
在reduce的过程中，提前作数据打包（处理粒度较大），通信时间（数据传输成本）会较小

### 练习：回归分析

计算 $\hat{y}=X(X^{T}X)^{-1}X^{T}y$

In [18]:
np.random.seed(123)
n = 2000
p = 500
X = np.random.normal(size=(n, p))
y = np.random.normal(size=n)